In [1]:
import json
import os
import bz2
import io
from bz2 import BZ2File
import pandas as pd
import numpy as np
import re
from utils import *

pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)
pd.set_option('display.max_colwidth', 100)

In [2]:
files_dir = "../data/impresso-data"
archives = os.listdir(files_dir)
date = "1990"

* Extract interesting archives

In [3]:
regex = '(.*1989.*)|(.*198[0-1].*)'
gdl_ = extract_particular_files(archives, regex, 'GDL')      
jdg_ = extract_particular_files(archives, regex, 'JDG') 

* Extract content  and preprocessing

In [4]:
 df_all, df_gdl, df_jdg = run(gdl_, jdg_, date, files_dir)

GDL-1980.jsonl.bz2 : 49639 articles à extraire
GDL-1981.jsonl.bz2 : 55490 articles à extraire
GDL-1989.jsonl.bz2 : 69464 articles à extraire
JDG-1980.jsonl.bz2 : 49477 articles à extraire
JDG-1981.jsonl.bz2 : 52220 articles à extraire
JDG-1989.jsonl.bz2 : 71376 articles à extraire


/opt/anaconda3/lib/python3.7/site-packages/pandas/core/generic.py:5303: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[name] = value


,mag,date,page,ppage,text,length
0,GDL,1980-10-28,1,NaN,"PRESIDENTIELLES FRANÇAISES La saison des trouble-fête Gaie, amusante, presque primesautière : vo...",3556
1,GDL,1980-10-28,2,NaN,"HhPâfrP Les spectacles de la semaine Comme au plein de chaque saison, vient le moment des choix ...",3749
2,GDL,1980-10-28,3,NaN,Université de Lausanne : les nouveaux docteurs honoris causa Mme Helena Raskova Fille d'un père ...,5953
3,GDL,1980-10-28,4,NaN,L'Afrique du Sud a-t-elle changé dernièrement ? Cet exposé vous l'apprendra COMMUNIQUÉ PUBLICITA...,8212
4,GDL,1980-10-28,5,NaN,Centre autonome : la réponse dans quatre mois « Levez d'abord les inculpations de manifestants e...,3820


Length df: 134      437
68       371
67       359
210      315
209      272
        ... 
16718      1
10778      1
14872      1
18966      1
34545      1
Name: length, Length: 11297, dtype: int64


/opt/anaconda3/lib/python3.7/site-packages/pandas/core/generic.py:5303: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[name] = value


,mag,date,page,ppage,text,length
0,GDL,1980-10-28,1,NaN,"PRESIDENTIELLES FRANÇAISES La saison des trouble-fête Gaie, amusante, presque primesautière : vo...",3556
1,GDL,1980-10-28,2,NaN,"HhPâfrP Les spectacles de la semaine Comme au plein de chaque saison, vient le moment des choix ...",3749
2,GDL,1980-10-28,3,NaN,Université de Lausanne : les nouveaux docteurs honoris causa Mme Helena Raskova Fille d'un père ...,5953
3,GDL,1980-10-28,4,NaN,L'Afrique du Sud a-t-elle changé dernièrement ? Cet exposé vous l'apprendra COMMUNIQUÉ PUBLICITA...,8212
4,GDL,1980-10-28,5,NaN,Centre autonome : la réponse dans quatre mois « Levez d'abord les inculpations de manifestants e...,3820


Length df: 134      292
68       256
67       223
210      203
114      172
        ... 
9252       1
7267       1
35364      1
8980       1
14321      1
Name: length, Length: 8606, dtype: int64


/opt/anaconda3/lib/python3.7/site-packages/pandas/core/generic.py:5303: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[name] = value


,mag,date,page,ppage,text,length
0,JDG,1980-10-28,1,NaN,JA No 252 MARDI 28 OCTOBRE 1980 SUISSE : 70 ct. FRANCE : 2 FF BELGIQUE : 16 FB ESPAGNE : 30 PESE...,152
1,JDG,1980-10-28,2,NaN,Editorial Il n'existe pas de « gentil racisme » L'éditorial de Claude Monnier intitulé « Qui est...,3250
2,JDG,1980-10-28,3,NaN,"ZAMBIE COUp d'Etat manqué à Lusaka KErlNETH KAUNDA L saka, 27 (Reuter).-Le président Kenneth ~ d...",1895
3,JDG,1980-10-28,4,NaN,L'Afrique du Sud a-t-elle changé dernièrement ? Cet exposé vous l'apprendra Une réorganisation c...,8246
4,JDG,1980-10-28,5,NaN,"Reprise de la productivité aux USA v h'ngton, 27 (AFP )'-La productivité de \ as ml'l'des Etats-...",1385


Length df: 209      159
134      145
199      138
67       136
206      134
        ... 
33364      1
6855       1
15171      1
35669      1
6934       1
Name: length, Length: 9932, dtype: int64


* Keywords filtering

In [5]:
keywords_1990 = ['nucléaire','initiatives', 'initiative', 'centrale', 'centrales', 'atomique', 'atomiques', 'suisse', 'energie', 'votation', 'abandon', 'progressif']
#keywords_1986 = ['nucléaire', 'centrale', 'centrales', 'atomique', 'atomiques', 'energie', 'ukraine', 'Tchernobyl', '1986', 'coeur fondu']

unwanted = ['tableau hebdomadaire']#, '(\/(\s\d*)*\s)*']

cleaned_files_dir = "../data/filtered/"
archives = os.listdir(files_dir)

In [6]:
#df_all = load_corpus(cleaned_files_dir + "cleaned_all" + date +".json.bz2")
#df_gdl = load_corpus(cleaned_files_dir + "cleaned_gdl" + date +".json.bz2")
#df_jdg = load_corpus(cleaned_files_dir + "cleaned_jdg" + date +".json.bz2")



In [7]:
df_all_ = keywords_filtering(df_all, keywords_1990)
df_all_ = unwanted_filtering(df_all_, unwanted)

===KEYWORDS===


'After keyword filtered: '

(52973, 19)

=====UNWANTED====


'After keyword filtered: '

(52973, 19)

In [8]:
df_gdl_ = keywords_filtering(df_gdl, keywords_1990)
df_gdl_ = unwanted_filtering(df_gdl_, unwanted)

===KEYWORDS===


'After keyword filtered: '

(22892, 19)

=====UNWANTED====


'After keyword filtered: '

(22892, 19)

In [9]:
df_jdg_ = keywords_filtering(df_jdg, keywords_1990)
df_jdg_ = unwanted_filtering(df_jdg_, unwanted)

===KEYWORDS===


'After keyword filtered: '

(30081, 19)

=====UNWANTED====


'After keyword filtered: '

(30081, 19)

* Export iramutek

In [10]:
display(df_all_.head())
display(df_gdl_.head())
display(df_jdg_.head())

,mag,date,page,ppage,text,length,keywords,keyword_nucléaire,keyword_initiatives,keyword_initiative,keyword_centrale,keyword_centrales,keyword_atomique,keyword_atomiques,keyword_suisse,keyword_energie,keyword_votation,keyword_abandon,keyword_progressif
8,GDL,1980-10-28,9,NaN,""" ~? vtm ? r .-Cornet i _/ * i > ii. w- >. . ri- H "" ..,. |., ""v ; rl >. r : j ;; ? no- no- ; q ...",764,2,0,0,0,0,0,0,0,2,0,0,0,0
9,GDL,1980-10-28,10,NaN,SOLVALOR 61 Fonds de placement immobilier so va 61 or Paiement du coupon N° 22 A partir du 31 oc...,960,4,0,0,0,0,0,0,0,4,0,0,0,0
11,GDL,1980-10-28,12,NaN,"PROTECTION DE L'ENVIRONNEMENT Lacs suisses : le poisson comestible se fait rare Berne, 27 (ATS)....",2594,2,0,0,0,0,0,0,0,2,0,0,0,0
12,GDL,1980-10-28,13,NaN,Alcoolisme au travail : la fin d'un tabou DEUX JOURNÉES D'INFORMATION D'ÉTUDES A BÂLE (I) Il y a...,5476,3,0,0,0,0,0,0,0,3,0,0,0,0
15,GDL,1980-10-28,16,NaN,BOURSES SUISSE 24 4 M) 1250 460 575 1025 1010 g 200 1770 15713 308 g 265 g 1000 574 tco 160 2625...,35402,7,0,0,0,0,0,0,0,7,0,0,0,0


,mag,date,page,ppage,text,length,keywords,keyword_nucléaire,keyword_initiatives,keyword_initiative,keyword_centrale,keyword_centrales,keyword_atomique,keyword_atomiques,keyword_suisse,keyword_energie,keyword_votation,keyword_abandon,keyword_progressif
8,GDL,1980-10-28,9,NaN,""" ~? vtm ? r .-Cornet i _/ * i > ii. w- >. . ri- H "" ..,. |., ""v ; rl >. r : j ;; ? no- no- ; q ...",764,2,0,0,0,0,0,0,0,2,0,0,0,0
9,GDL,1980-10-28,10,NaN,SOLVALOR 61 Fonds de placement immobilier so va 61 or Paiement du coupon N° 22 A partir du 31 oc...,960,4,0,0,0,0,0,0,0,4,0,0,0,0
11,GDL,1980-10-28,12,NaN,"PROTECTION DE L'ENVIRONNEMENT Lacs suisses : le poisson comestible se fait rare Berne, 27 (ATS)....",2594,2,0,0,0,0,0,0,0,2,0,0,0,0
12,GDL,1980-10-28,13,NaN,Alcoolisme au travail : la fin d'un tabou DEUX JOURNÉES D'INFORMATION D'ÉTUDES A BÂLE (I) Il y a...,5476,3,0,0,0,0,0,0,0,3,0,0,0,0
15,GDL,1980-10-28,16,NaN,BOURSES SUISSE 24 4 M) 1250 460 575 1025 1010 g 200 1770 15713 308 g 265 g 1000 574 tco 160 2625...,35402,7,0,0,0,0,0,0,0,7,0,0,0,0


,mag,date,page,ppage,text,length,keywords,keyword_nucléaire,keyword_initiatives,keyword_initiative,keyword_centrale,keyword_centrales,keyword_atomique,keyword_atomiques,keyword_suisse,keyword_energie,keyword_votation,keyword_abandon,keyword_progressif
5,JDG,1980-10-28,6,NaN,SUISSE GENÈVI :: a 1 Hcha. ç ; e all. lsu ! Sse p alu $ u 1 sse fi DaLols ~ n oaloin Op charmil....,36673,5,0,0,0,0,0,0,0,5,0,0,0,0
9,JDG,1980-10-28,10,NaN,AVIS FINANCIERS 11111 50 va or 61 SOLVALOR 61 Fonds de placement immobilier Paiement du coupon N...,973,4,0,0,0,0,0,0,0,4,0,0,0,0
12,JDG,1980-10-28,13,NaN,"/ Alcoolisme au travail : la fin d'un tabou Il Y a, estime-t-on, quelque cent cinquante mille al...",5567,3,0,0,0,0,0,0,0,3,0,0,0,0
13,JDG,1980-10-28,14,NaN,Jean-Claude KERNI'SAN Directeur de PROMOVENTE SA'répond à 3 questions que vous pourriez lui pose...,2650,2,0,0,0,0,0,0,0,2,0,0,0,0
24,JDG,1980-10-28,2,NaN,"RÉACTION L'homme . ,. craint et meprise ""tiQJJ} Q 1 Jl : J 1 I' 1 ., lgn ,,, 11 Çl ture L ~ 'dep...",1990,3,0,0,0,0,0,0,0,3,0,0,0,0


In [11]:
creating_iramutek_file(df_all_, "../data/iramutek/corpus_all" + date +".txt")
creating_iramutek_file(df_jdg_, "../data/iramutek/corpus_jdg" + date +".txt")
creating_iramutek_file(df_gdl_, "../data/iramutek/corpus_gdl" + date +".txt")